## Decision Tree

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# Load the Excel file
nifty50_excel_file = 'combined_stock_data_single_sheet.xlsx'
xls = pd.ExcelFile(nifty50_excel_file)

# Define features (X)
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume',
            'Previous_Close', '5SMA', '10SMA', '20SMA', '50SMA', '100SMA', '200SMA',
            '5EMA', '10EMA', '20EMA', 'MACD', 'MACD_signal', 'RSI', 'PSAR',
            'vortex_indicator', 'Upper_Band', 'Lower_Band', 'ATR5', 'ATR10',
            'ATR20', 'ATR50', 'Stoch_Signal', 'Stoch', 'WR', 'TSI', 'ADX', 'VWAP',
            'Daily_Return', 'Cumulative_Return', 'ROC5', 'ROC10', 'ROC20', 'ROC50',
            'ROC100', 'ROC200', 'CMF', 'Daily_Log_Return']

# Create an empty DataFrame to store results
all_results = pd.DataFrame(columns=['Company', 'RMSE', 'MSE', 'MAE', 'R-squared', 'MAPE'])

# Define the number of days to shift for future prediction (e.g., 10 days ahead)
days_to_shift = 10

# Loop through each sheet (company) in the Excel file
for sheet_name in xls.sheet_names:
    print(f"Processing data for {sheet_name}...")

    try:
        df = pd.read_excel(nifty50_excel_file, sheet_name=sheet_name)

        # Shift the 'Close' column to create the target variable for 10 days ahead prediction
        df[f'Close_{days_to_shift}_Days_Ahead'] = df['Close'].shift(-days_to_shift)

        # Drop rows with missing values (last rows where target is NaN)
        df = df.dropna()

        # Define the target variable (y)
        target = f'Close_{days_to_shift}_Days_Ahead'

        X = df[features]
        y = df[target]

        # Calculate the split point
        split_point = int(len(X) * 0.8)  # 80% of the data for training, 20% for testing

        X_train, X_test = X[:split_point], X[split_point:]
        y_train, y_test = y[:split_point], y[split_point:]

        # Standardize the features
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Create a Decision Tree Regressor model
        decision_tree_model = DecisionTreeRegressor()

        # Train the model
        decision_tree_model.fit(X_train, y_train)

        # Predict the target variable on the test data
        y_pred = decision_tree_model.predict(X_test)

        # Calculate evaluation metrics
        rmse = math.sqrt(mean_squared_error(y_test, y_pred))
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r_squared = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

        # Store results in the DataFrame
        result = pd.DataFrame({
            'Company': [sheet_name],
            'RMSE': [rmse],
            'MSE': [mse],
            'MAE': [mae],
            'R-squared': [r_squared],
            'MAPE': [mape]
        })

        all_results = all_results.append(result, ignore_index=True)

        print(f"Data for {sheet_name} processed.")

    except Exception as e:
        print(f"Error processing data for {sheet_name}: {str(e)}")

# Save the results to a CSV file
all_results.to_csv(f'decision_tree_results_{days_to_shift}_days_ahead.csv', index=False)

print(f'Results saved to decision_tree_results_{days_to_shift}_days_ahead.csv')
